# Here, I am creating a new column in the goalie_hist_model_ready DF that creates a Goalie Score metric for analysis. 

In [6]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


In [7]:
#Import goalie-specific, model ready csv that alrerady has the Saves_percentages and expected total points from the earlier 2 models - this is the master.

goalie_hist_model_ready = pd.read_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/total_point_predictions_df_master.csv")
#Ensure we can read the whole dataframe, without "..."
pd.set_option("display.max_rows", None)
goalie_hist_model_ready = goalie_hist_model_ready.drop(columns = ['Unnamed: 0'])
goalie_hist_model_ready.head()


,id,first_name,second_name,team,element_type,code,element_code,season,minutes,goals_scored,...,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,Saves_Percentage,start_cost,end_cost,Predicted_End_Cost,total_points,Predicted_Total_Points
0,15,David,Raya Martin,1,1,154561,154561,2021,2160,0,...,0.00,0.00,0.00,0.00,74.285714,45,44,45.376,95,90.344
1,15,David,Raya Martin,1,1,154561,154561,2022,3420,0,...,0.11,0.12,0.23,50.12,77.000000,45,48,49.154,166,160.620
2,15,David,Raya Martin,1,1,154561,154561,2023,2880,0,...,0.00,0.04,0.04,22.51,66.197183,50,53,54.892,135,135.108
3,73,Norberto,Murara Neto,1,1,69752,69752,2022,2385,0,...,0.00,0.02,0.02,44.70,71.942446,45,45,44.196,93,93.902
4,73,Norberto,Murara Neto,1,1,69752,69752,2023,2880,0,...,0.00,0.01,0.01,49.66,67.455621,45,46,46.408,110,110.714


In [8]:
# New calculation for Goalie Score

goalie_hist_model_ready.columns

Index(['id', 'first_name', 'second_name', 'team', 'element_type', 'code',
       'element_code', 'season', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts',
       'expected_goals', 'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'Saves_Percentage', 'start_cost', 'end_cost',
       'Predicted_End_Cost', 'total_points', 'Predicted_Total_Points'],
      dtype='object')

In [17]:
goalie_score_1 = goalie_hist_model_ready
goalie_score_1["Goalie_Score"]= (goalie_hist_model_ready['end_cost'] + goalie_hist_model_ready['bonus'] + goalie_hist_model_ready['ict_index']) * (goalie_hist_model_ready['Saves_Percentage']/100)
goalie_hist_model_ready.head()


,id,first_name,second_name,team,element_type,code,element_code,season,minutes,goals_scored,...,expected_assists,expected_goal_involvements,expected_goals_conceded,Saves_Percentage,start_cost,end_cost,Predicted_End_Cost,total_points,Predicted_Total_Points,Goalie_Score
0,15,David,Raya Martin,1,1,154561,154561,2021,2160,0,...,0.00,0.00,0.00,74.285714,45,44,45.376,95,90.344,81.045714
1,15,David,Raya Martin,1,1,154561,154561,2022,3420,0,...,0.12,0.23,50.12,77.000000,45,48,49.154,166,160.620,142.681000
2,15,David,Raya Martin,1,1,154561,154561,2023,2880,0,...,0.04,0.04,22.51,66.197183,50,53,54.892,135,135.108,66.263380
3,73,Norberto,Murara Neto,1,1,69752,69752,2022,2385,0,...,0.02,0.02,44.70,71.942446,45,45,44.196,93,93.902,91.223022
4,73,Norberto,Murara Neto,1,1,69752,69752,2023,2880,0,...,0.01,0.01,49.66,67.455621,45,46,46.408,110,110.714,99.497041


In [20]:
#create X & y variables
X = goalie_hist_model_ready.drop(columns = ['Goalie_Score','id', 'first_name', 'second_name', 'team', 'element_type', 'code',
       'element_code', 'season'])
y = goalie_hist_model_ready['Goalie_Score']

#test-training split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#create model
model_lr = LinearRegression()

model_lr.fit(X_train, y_train)
print("Train model score: ", model.score(X_train, y_train))
print("Test model score: ", model.score(X_test, y_test))

Train model score:  0.9990342807862905
Test model score:  0.9967032161066917


In [21]:
#Make predictions

prediction1 = model_lr.predict(X_test)

#Evaluate models with mse and r2

mse = mean_squared_error(y_test, prediction1) # how close are the predicted values to actual values via the squared differences between expected and real
r2 = r2_score(y_test, prediction1) # r2 - how well do the indep variables explain the variation in the dep var? ) 0 is a perfect model, the larger the nuber, the worse the model is performing

print(f"All Features (Saves_Percentage = y):")
print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

All Features (Saves_Percentage = y):
mean squared error (MSE): 3.5945299411193172
R-squared (R2): 0.9967032161066917


In [25]:
#Run the model to get the predicted Goalie Score

predicted_goalie_score = goalie_hist_model_ready.drop(columns = ['Goalie_Score','id', 'first_name', 'second_name', 'team', 'element_type', 'code',
       'element_code', 'season'])
lr_goalie_score = model_lr.predict(predicted_goalie_score)
print(lr_goalie_score)

[ 79.92716422 142.73443217  65.99535777  90.75080425  98.60026859
  22.49306546  26.2080463   55.1161023  141.11963744  93.72322028
 108.48480578 103.92272435  45.39171046  16.59804652  27.7816885
  40.00468883  30.3483318   17.69711883  37.71852698  37.34124974
  89.01755565  57.62511293  42.67125135  46.02562616  96.27887488
 101.01828714  25.2146905   55.86308494  46.2294625   74.34464732
  32.1444321   73.21389528  77.94387561  96.31253986  61.64551159
  66.40685281 106.61316517  61.17121639  67.7490009   59.25599224
  -0.57845707  47.96483614  44.62118987  24.83336758 108.87441842
 105.60377841  96.40997886  84.85254327  89.36981352 101.2839084
 106.67356825 122.49099325  35.44084713  24.57387134  40.70857184
  34.83032343  47.95333291  46.91489362  33.05717536  32.48297093
  11.44635955  85.15985496  49.97842208  29.68818443  43.31220995
  96.20223008 104.48804704  92.42571987  29.71626765 130.61758138
 116.86648435  86.86699834  44.38244617  27.91413969  34.65106449
  71.0531708

In [23]:
#Random Forest
#create X & y variables
X = goalie_hist_model_ready.drop(columns=['Goalie_Score','id', 'first_name', 'second_name', 'team', 'element_type', 'code',
       'element_code', 'season'])
y = goalie_hist_model_ready['Goalie_Score']

#test-training split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#create & train model
random_forest = RandomForestRegressor(n_estimators=500, random_state=42).fit(X_train, y_train)

# Evaluate the model
print(f"Training Score: {random_forest.score(X_train, y_train)}")
print(f"Testing Score: {random_forest.score(X_test, y_test)}")

Training Score: 0.9961666420263626
Testing Score: 0.9655145235654327


In [24]:
# Feature Importance
feature_importances = random_forest.feature_importances_

feature_importances_df = pd.DataFrame(feature_importances, X.columns)


print(feature_importances_df.sort_values(by=0, ascending=False))

                                   0
ict_index                   0.667555
influence                   0.245516
Saves_Percentage            0.023679
bps                         0.016121
saves                       0.013130
total_points                0.006460
Predicted_Total_Points      0.005570
bonus                       0.005543
Predicted_End_Cost          0.004798
end_cost                    0.002295
goals_conceded              0.001886
clean_sheets                0.001638
minutes                     0.001518
yellow_cards                0.001151
creativity                  0.000700
expected_goals_conceded     0.000399
starts                      0.000384
start_cost                  0.000370
threat                      0.000248
own_goals                   0.000233
expected_goal_involvements  0.000229
penalties_saved             0.000221
expected_assists            0.000134
assists                     0.000072
red_cards                   0.000054
expected_goals              0.000050
g

In [26]:
#Run the model to get the predicted Goalie Score from the RFR model

rfr_predicted_goalie_score = goalie_hist_model_ready.drop(columns = ['Goalie_Score','id', 'first_name', 'second_name', 'team', 'element_type', 'code',
       'element_code', 'season'])
rfr_Goalie_Score = random_forest.predict(predicted_goalie_score)
print(rfr_Goalie_Score)

[ 81.25092201 138.81234608  64.59236245  89.05618454 100.77365706
  23.3852781   25.89721639  53.28306318 136.95372562  89.85846413
 109.19434696  99.51740848  45.7704981   17.49854711  27.72390514
  43.12984022  29.36812584  17.47983579  44.25271899  41.67970817
  88.99268854  60.85775944  44.53367651  44.82266075  97.71387224
 101.9805755   24.03303798  53.51040098  47.50021841  76.69147697
  41.35320997  72.10277976  80.5006174   97.71599532  60.04831237
  64.53257181 104.12286972  58.42454792  66.72177699  58.51438115
   6.50967072  51.02396838  37.47592661  26.29509663 109.49240005
 112.25023203  94.808356    85.88212435  88.78479372 101.52061876
 107.21750633 116.21952781  35.14577164  26.67788159  36.28007911
  35.02085056  39.24187533  44.57286072  35.43754699  34.00805185
  14.59371431  87.41575189  52.74655493  28.54512506  40.0146003
  92.33685463 104.51889656  93.64188429  28.39573475 128.81338572
 117.55155874  83.61094981  43.91922239  29.65961281  34.79326639
  73.070305

# The Linear Regression model has a 99.67% testing score vs the RFR's of 96.55%, so we will use the LR for the model

In [29]:
# Create new dataframe with expected Goalie Score

goalie_score_prediction_df_master = goalie_hist_model_ready
goalie_score_prediction_df_master['Predicted_Goalie_Score'] = lr_goalie_score
goalie_score_prediction_df_master = goalie_score_prediction_df_master
goalie_score_prediction_df_master.head(15)

goalie_score_prediction_df_master.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/goalie_score_prediction_df_master.csv")

#total_points_predictions_df_master.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/total_point_predictions_df_master.csv")
#total_points_predictions_df_master.head(10)